# Run NoteBook
To run the notebook you will need Jupyter notebook, go to Cell and select Run All Below. This will run the notebook for the today's data.

In [85]:
import numpy as np
import pandas as pd
import re
import datetime

# Assignment

This assignment is to make a simple Surf Report application by using the buoy data provided by NOAA. The data feed is a simple tab delimited file that contains wind & swell data for the last 45 days.

To simplify this assignment, we will be focusing on a single buoy off the coast of Oregon. The data feed for this buoy can be found here:

https://www.ndbc.noaa.gov/data/realtime2/46029.spec

The description for what each column means can be found here: https://www.ndbc.noaa.gov/waveobs.shtml

# Data

In [86]:
data = 'https://www.ndbc.noaa.gov/data/realtime2/46029.spec'
surf_table = pd.read_csv(data, sep='\s+', delimiter=' ', encoding="utf-8", skipinitialspace=True)

In [87]:
surf_table =surf_table.drop(0)

Initial Data Table

In [88]:
surf_table.head()

,#YY,MM,DD,hh,mm,WVHT,SwH,SwP,WWH,WWP,SwD,WWD,STEEPNESS,APD,MWD
1,2019,03,24,06,00,4.3,4.2,14.8,0.4,3.7,W,SW,SWELL,11.1,268
2,2019,03,24,05,00,3.9,3.9,14.8,0.4,3.4,W,SSW,SWELL,11.1,270
3,2019,03,24,04,00,3.4,3.4,12.9,0.4,3.6,W,S,SWELL,10.7,272
4,2019,03,24,03,00,3.3,3.3,14.8,0.4,3.3,W,S,SWELL,10.7,272
5,2019,03,24,02,00,3.7,3.7,13.8,0.3,3.7,W,SSW,SWELL,11.2,268


In [89]:
short_table = surf_table.drop(article_read.iloc[:,3:6,], axis=1)

In [90]:
short_table = short_table.drop(['STEEPNESS', 'APD', 'MWD'], axis=1)

Today's Date

In [91]:
d = datetime.date.today().strftime('%Y-%m-%d')
d

'2019-03-23'

In [92]:
short_table = short_table.rename(columns={'#YY': 'YY'})

In [93]:
date = (short_table.YY + '-' + short_table.MM + '-' + short_table.DD)
short_table['Date'] = date

In [94]:
data_table = short_table.drop(short_table.columns[[0,1,2]], axis=1)
data_table = data_table[data_table['Date']==d]
data_table

,SwH,SwP,WWH,WWP,SwD,WWD,Date
8,3.6,13.8,0.3,3.7,W,SSW,2019-03-23
9,3.3,16.0,0.3,3.7,W,SSW,2019-03-23
10,3.7,16.0,0.4,3.6,W,SSW,2019-03-23
11,2.8,14.8,0.4,3.6,WNW,S,2019-03-23
12,3.5,14.8,0.5,3.7,W,S,2019-03-23
13,3.7,14.8,0.6,5.0,W,SW,2019-03-23
14,3.3,13.8,0.8,5.0,W,SW,2019-03-23
15,3.0,12.9,0.8,3.8,W,SW,2019-03-23
16,3.4,14.8,0.6,4.0,W,SSW,2019-03-23
17,3.5,14.8,0.5,4.0,W,SSW,2019-03-23


Change data from object to float

In [95]:
data_table['SwH'] = data_table.SwH.astype(float)
data_table['SwP'] = data_table.SwP.astype(float)
data_table['WWH'] = data_table.WWH.astype(float)
data_table['WWP'] = data_table.WWP.astype(float)

In [96]:
data_table.dtypes

SwH     float64
SwP     float64
WWH     float64
WWP     float64
SwD      object
WWD      object
Date     object
dtype: object

In [97]:
wave = data_table.SwH * data_table.SwP
data_table['Wave'] = wave

In [98]:
surf_table.WWD.unique()

array(['SW', 'SSW', 'S', 'ESE', 'SSE', 'SE', 'E', 'WSW', 'ENE', 'N', 'W',
       'NNE', 'WNW', 'NE', 'NW', 'NNW'], dtype=object)

# Testing Setup

The test is setup inside the function and located between the ''' qoutes marked in red. You can call the function and provide the perameter and expected outcome. In section 'Run Test' at the end of the notebook, the test will generate and provide the result.

Example:

''' This is a test: >>> scoreWwD('E') 5 '''

In [99]:
def scoreWWD(column): 
    '''
    This is a test:
    >>> scoreWWD('E')
    5
    
    This is a test:
    >>> scoreWWD('NE')
    4

    This is a test:
    >>> scoreWWD('SE')
    4

    This is a test:
    >>> scoreWWD('S')
    3

    This is a test:
    >>> scoreWWD('SSW')
    1
    '''
    
    if column == 'E' :
        score = 5
    elif column =='NE' :
        score = 4
    elif column == 'SE' :
        score = 4
    elif column == 'S' :
        score = 3
    else:
        score =1
    return score

In [100]:
def scoreWave(column): 
    '''
    This is a test:
    >>> scoreWave(31)
    5
    
    This is a test:
    >>> scoreWave(30)
    4

    This is a test:
    >>> scoreWave(11)
    2

    This is a test:
    >>> scoreWave(5)
    1
    '''
    
    if column > 30 :
        score = 5
    elif column >= 25 :
        score = 4
    elif column >= 20 :
        score = 3
    elif column >= 11 :
        score = 2
    else:
        score =1
    return score 

In [101]:
def scoreSwp(column):
    '''
    This is a test:
    >>> scoreSwp(16)
    5
    
    This is a test:
    >>> scoreSwp(15)
    4
    
    This is a test:
    >>> scoreSwp(10)
    4
    
    This is a test:
    >>> scoreSwp(5)
    1
    '''
        
    if column >= 16 :
        score = 5
    elif column >= 12 :
        score = 4
    elif column >= 10 :
        score = 4
    else:
        score =1
    return score 

In [102]:
data_table['scoreWave'] = data_table['Wave'].map(scoreWave)
data_table['scoreSwP'] = data_table['SwP'].map(scoreSwp)
data_table['scoreWWD'] = data_table['WWD'].map(scoreWWD)

In [103]:
data_length = len(data_table)-1
data_length

23

In [104]:
data_table

,SwH,SwP,WWH,WWP,SwD,WWD,Date,Wave,scoreWave,scoreSwP,scoreWWD
8,3.6,13.8,0.3,3.7,W,SSW,2019-03-23,49.68,5,4,1
9,3.3,16.0,0.3,3.7,W,SSW,2019-03-23,52.80,5,5,1
10,3.7,16.0,0.4,3.6,W,SSW,2019-03-23,59.20,5,5,1
11,2.8,14.8,0.4,3.6,WNW,S,2019-03-23,41.44,5,4,3
12,3.5,14.8,0.5,3.7,W,S,2019-03-23,51.80,5,4,3
13,3.7,14.8,0.6,5.0,W,SW,2019-03-23,54.76,5,4,1
14,3.3,13.8,0.8,5.0,W,SW,2019-03-23,45.54,5,4,1
15,3.0,12.9,0.8,3.8,W,SW,2019-03-23,38.70,5,4,1
16,3.4,14.8,0.6,4.0,W,SSW,2019-03-23,50.32,5,4,1
17,3.5,14.8,0.5,4.0,W,SSW,2019-03-23,51.80,5,4,1


In [105]:
totalscore = data_table['scoreWWD'].sum()/(len(data_table)-1)
totalscore

1.3043478260869565

In [106]:
day_score = round(totalscore)
day_score

1.0

In [107]:
score = data_table[['scoreWWD','scoreWave','scoreSwP']].mean()
score

scoreWWD     1.250000
scoreWave    5.000000
scoreSwP     4.291667
dtype: float64

In [108]:
day_score = score.sum()/3
day_score

3.5138888888888893

# Run Test

In [109]:
import doctest
doctest.testmod(verbose=True)

Trying:
    scoreSwp(16)
Expecting:
    5
ok
Trying:
    scoreSwp(15)
Expecting:
    4
ok
Trying:
    scoreSwp(10)
Expecting:
    4
ok
Trying:
    scoreSwp(5)
Expecting:
    1
ok
Trying:
    scoreWWD('E')
Expecting:
    5
ok
Trying:
    scoreWWD('NE')
Expecting:
    4
ok
Trying:
    scoreWWD('SE')
Expecting:
    4
ok
Trying:
    scoreWWD('S')
Expecting:
    3
ok
Trying:
    scoreWWD('SSW')
Expecting:
    1
ok
Trying:
    scoreWave(31)
Expecting:
    5
ok
Trying:
    scoreWave(30)
Expecting:
    4
ok
Trying:
    scoreWave(11)
Expecting:
    2
ok
Trying:
    scoreWave(5)
Expecting:
    1
ok
Trying:
    total('scoreWWD') 
Expecting:
    36
**********************************************************************
File "__main__", line 4, in __main__.total
Failed example:
    total('scoreWWD') 
Expected:
    36
Got:
    30
Trying:
    total('scoreWave') 
Expecting:
    80
**********************************************************************
File "__main__", line 8, in __main__.total
Failed ex

TestResults(failed=5, attempted=19)

The test is inside the function marked by ''' , this is space sensative so data must match the alighment.

In [110]:
data_length

23

In [111]:
data_table.apply(np.sum, axis=0)

SwH                                                       82.2
SwP                                                      358.7
WWH                                                       14.2
WWP                                                      100.3
SwD                               WWWWNWWWWWWWWWWWWWSWWWWWWWWW
WWD          SSWSSWSSWSSSWSWSWSSWSSWSWSWSWSSWSSSWSWSWSWSWSS...
Date         2019-03-232019-03-232019-03-232019-03-232019-0...
Wave                                                    1229.4
scoreWave                                                  120
scoreSwP                                                   103
scoreWWD                                                    30
dtype: object

# Today's Condition

In [112]:
data_table

,SwH,SwP,WWH,WWP,SwD,WWD,Date,Wave,scoreWave,scoreSwP,scoreWWD
8,3.6,13.8,0.3,3.7,W,SSW,2019-03-23,49.68,5,4,1
9,3.3,16.0,0.3,3.7,W,SSW,2019-03-23,52.80,5,5,1
10,3.7,16.0,0.4,3.6,W,SSW,2019-03-23,59.20,5,5,1
11,2.8,14.8,0.4,3.6,WNW,S,2019-03-23,41.44,5,4,3
12,3.5,14.8,0.5,3.7,W,S,2019-03-23,51.80,5,4,3
13,3.7,14.8,0.6,5.0,W,SW,2019-03-23,54.76,5,4,1
14,3.3,13.8,0.8,5.0,W,SW,2019-03-23,45.54,5,4,1
15,3.0,12.9,0.8,3.8,W,SW,2019-03-23,38.70,5,4,1
16,3.4,14.8,0.6,4.0,W,SSW,2019-03-23,50.32,5,4,1
17,3.5,14.8,0.5,4.0,W,SSW,2019-03-23,51.80,5,4,1


# Score for the Day

In [113]:
scores = data_table[['scoreWWD','scoreWave','scoreSwP']].mean()
scores

scoreWWD     1.250000
scoreWave    5.000000
scoreSwP     4.291667
dtype: float64

In [114]:
final_score = scores.sum()/3
final_score

3.5138888888888893